# Import Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ['NEPTUNE_API_TOKEN']="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI5NDc0ZmNhNi0wODFlLTRhYTktYjgwZS01MWJkMDMxNWJhNTAifQ=="
os.environ['NEPTUNE_PROJECT']="guilhermemg/icao-nets-training-2"
os.environ['NEPTUNE_NOTEBOOK_ID']="98a391a1-c710-40bd-aaf4-42c31862cbbe"
os.environ['NEPTUNE_NOTEBOOK_PATH']="training/exec_nas_experiment.ipynb"

In [3]:
import os
# disable tensorflow log level infos
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # show only errors

import sys
import pandas as pd

if '..' not in sys.path:
    sys.path.insert(0, '..')

In [4]:
from src.m_utils import constants as cts
from src.base.data_loaders.data_loader import DLName
from src.base.gt_loaders.gt_names import GTName
from src.exp_runner import ExperimentRunner

from src.base.experiment.dataset.dataset import Dataset
from src.base.experiment.evaluation.model_evaluator import DataSource, DataPredSelection
from src.base.experiment.training.base_models import BaseModel
from src.base.experiment.training.optimizers import Optimizer

from src.m_utils.mtl_approach import MTLApproach
from src.m_utils.nas_mtl_approach import NAS_MTLApproach

 ==> Restrict GPU memory growth: True


# Start Network runner

In [5]:
DATASET = Dataset.FVC_ICAO
APPROACH = NAS_MTLApproach.APPROACH_2

kwargs = { 
    'use_neptune': True,
    'exp_params' : {
        'name': 'neural_arch_search',
        'description': 'NAS with Approach 2 Fixed',
        'tags': ['nas_approach_2_fixed', 'benchmark', f'{DATASET.value["name"]}', 'controller_use_predictor'],
        'src_files': ["../src/**/*.py"]
    },
    'properties': {
        'approach': APPROACH,
        'benchmarking': {
            'use_benchmark_data': False,
            'dataset': DATASET
        },
        'icao_data': {
            'icao_gt': {
                'use_gt_data': True,
                'gt_names': {
                    'train_validation': [],
                    'test': [],
                    'train_validation_test': [GTName.FVC]
                },
            },
            'icao_dl': {
                'use_dl_data': False,
                'tagger_model': None
            },
            'reqs': Dataset.FVC_ICAO.value['tasks'],
            'aligned': False
        },
        'balance_input_data': False,
        'train_model': True,
        'save_trained_model': True,
        'exec_nas': True,
        'orig_model_experiment_id': '',
        'sample_training_data': False,
        'sample_prop': 1.0
    },
    'nas_params': {
        #'max_blocks_per_branch': 5,
        #'n_child_epochs': N_CHILD_EPOCHS,
        #'controller_epochs': CONTROLLER_EPOCHS,
        #'controller_batch_size': 64,
        #'n_trials': N_TRIALS,

        'controller_sampling_epochs': 2,
        'samples_per_controller_epochs': 3,
        'controller_training_epochs': 5,
        'architecture_training_epochs': 2,
        'controller_loss_alpha': 0.9
    },
    'controller_params': {
        'controller_lstm_dim': 100,
        'controller_optimizer': Optimizer.ADAM,
        'controller_learning_rate': 0.01,
        'controller_decay': 0.1,
        'controller_momentum': 0.0,
        'controller_use_predictor': True
    },
    'mlp_params': {
        'max_architecture_length': 5,
        'min_task_group_size': 3,
        'mlp_base_model': BaseModel.MOBILENET_V2,
        'mlp_n_epochs': 50,
        'mlp_batch_size': 64,
        'mlp_early_stopping': 50,
        'mlp_optimizer': Optimizer.ADAMAX,
        'mlp_learning_rate': 1e-3,
        'mlp_decay': 0.0,
        'mlp_momentum': 0.0,
        'mlp_dropout': 0.3,
        'mlp_loss_function': 'sparse_categorical_crossentropy',
        'mlp_one_shot': False
    }
}


runner = ExperimentRunner(**kwargs)

-------------------- Init ExperimentRunner -------------------
---------------------------
Parent Process ID: 257557
Process ID: 815852
---------------------------
-----
Use Neptune:  True
-----
-------------------
Args: 
{'controller_params': {'controller_decay': 0.1,
                       'controller_learning_rate': 0.01,
                       'controller_lstm_dim': 100,
                       'controller_momentum': 0.0,
                       'controller_optimizer': <Optimizer.ADAM: 'Adam'>,
                       'controller_use_predictor': True},
 'exp_params': {'description': 'NAS with Approach 2 Fixed',
                'name': 'neural_arch_search',
                'src_files': ['../src/**/*.py'],
                'tags': ['nas_approach_2_fixed',
                         'benchmark',
                         'fvc_icao',
                         'controller_use_predictor']},
 'mlp_params': {'max_architecture_length': 5,
                'min_task_group_size': 3,
                'm

# Load Data

In [6]:
runner.load_training_data()

-------------------- load training data -------------------
Loading data
Loading GT FVC - TRAIN split...
..Ignoring 0 empty label values
Input data.shape: (4928, 26)
Loading GT FVC - VALIDATION split...
..Ignoring 0 empty label values
Input data.shape: (547, 26)
Loading GT FVC - TEST split...
..Ignoring 0 empty label values
Input data.shape: (288, 26)
Data loaded


In [7]:
runner.train_data.head()

,origin,img_name,mouth,rotation,l_away,eyes_closed,close,hat,dark_glasses,frames_heavy,...,sh_head,blurred,ink_mark,skin_tone,washed_out,pixelation,hair_eyes,background,red_eyes,aligned
0,fvc,/home/guilherme/data1/Dropbox/Link to Desktop/...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,False
1,fvc,/home/guilherme/data1/Dropbox/Link to Desktop/...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,False
2,fvc,/home/guilherme/data1/Dropbox/Link to Desktop/...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,False
3,fvc,/home/guilherme/data1/Dropbox/Link to Desktop/...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,False
4,fvc,/home/guilherme/data1/Dropbox/Link to Desktop/...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,False


# Data Generators

In [8]:
runner.setup_data_generators()

-------------------- setup data generators -------------------
Starting data generators
Found 4928 validated image filenames.
Found 547 validated image filenames.
Found 288 validated image filenames.
TOTAL: 5763

Logging class indices
 .. MTL model not logging class indices!

Logging class labels
 COMPLIANT label: 1
 NON_COMPLIANT label: 0
 DUMMY label: -1
 DUMMY_CLS label: 2
 NO_ANSWER label: -99


# Setup Experiment

In [9]:
runner.setup_experiment()

-------------------- create experiment -------------------
Setting up neptune experiment
Neptune experiment setup done!


# Labels Distribution

In [10]:
%%capture
runner.summary_labels_dist()

# Neural Architecture Search

In [11]:
best_archs_list = runner.run_neural_architecture_search_v2()

-------------------- run neural architecture search -------------------
----
Checking model existence locally...
Training a new model! Not checking model existence
----
------------------------------
Checking GPU availability
 ..GPU is available!
------------------------------
------------------------------------------------------------------
                       CONTROLLER EPOCH: 0
------------------------------------------------------------------
GENERATING ARCHITECTURE SAMPLES...
------------------------------------------------------
Sequence: [16, 1, 3, 3, 20]
 .Decoded seq: {'n_denses_3': 5, 'n_denses_0': 1, 'n_denses_2': 1}
 ..invalid sequence: less than 4 task groups!
Sequence: [3, 5, 14, 17, 20]
 .Decoded seq: {'n_denses_2': 1, 'n_denses_0': 5, 'n_denses_1': 4, 'n_denses_3': 5}
  ..valid sequence!
Sequence: [12, 8, 11, 5, 20]
 .Decoded seq: {'n_denses_3': 5, 'n_denses_2': 3, 'n_denses_0': 2}
 ..invalid sequence: less than 4 task groups!
Sequence: [12, 6, 13, 13, 20]
 .Decoded

/home/guilherme/data2/anaconda3/envs/icao_nets_training/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '



Epoch 00002: val_loss improved from 3.69146 to 3.39767, saving model to training_ckpt/best_model.hdf5
..Loading best model
..Checkpoint weights loaded
Testing Trained Model
Predicting labels....
9/9 [==============================] - 33s 4s/step
Prediction finished!


/home/guilherme/data2/anaconda3/envs/icao_nets_training/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:803: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn("No negative samples in y_true, "


  Task  0: mouth           | EER_mean: 0.28350 | EER_interp: 0.28220 | ACC: 0.71850
  Task  1: rotation        | EER_mean: 0.36500 | EER_interp: 0.36420 | ACC: 0.63620
  Task  2: l_away          | EER_mean: 0.22420 | EER_interp: 0.22600 | ACC: 0.77330
  Task  3: eyes_closed     | EER_mean: 0.07660 | EER_interp: 0.07380 | ACC: 0.92140
  Task  4: close           | EER_mean: 0.26050 | EER_interp: 0.00000 | ACC: 0.01100
  Task  5: hat             | EER_mean: 0.44640 | EER_interp: 0.00000 | ACC: 0.17000
  Task  6: dark_glasses    | EER_mean: 0.08800 | EER_interp: 0.00000 | ACC: 0.09510
  Task  7: frames_heavy    | EER_mean: 0.50000 | EER_interp: 0.00000 | ACC: 0.00550
  Task  8: frame_eyes      | EER_mean: 0.15940 | EER_interp: 0.15940 | ACC: 0.84280
  Task  9: flash_lenses    | EER_mean: 0.11520 | EER_interp: 0.11070 | ACC: 0.88670
  Task 10: veil            | EER_mean: 0.49230 | EER_interp: 0.00000 | ACC: 0.06030
  Task 11: reflection      | EER_mean: 0.21990 | EER_interp: 0.21750 | ACC: 

# Create Model with Best Architecture Found

In [12]:
#best_arch = {'n_denses_0':2,'n_denses_1':2,'n_denses_2':2,'n_denses_3':2}
best_arch = best_archs_list[0]['Decoded architecture']

In [13]:
runner.create_model(best_arch)

-------------------- create model -------------------
Creating model...
Model created


# Visualize Model

In [14]:
%%capture
runner.visualize_model(outfile_path=f"figs/nas/nas_model_{APPROACH.name}.png")

In [15]:
%%capture
runner.model_summary()

# Training Model

In [ ]:
runner.train_model()

-------------------- train model -------------------
Training MOBILENET_V2 network
 .. Not fine tuning base model...
  .. Total params: 4,283,886
  .. Trainable params: 2,025,902
  .. Non-trainable params: 2,257,984
Epoch 1/50
77/77 [==============================] - 355s 5s/step - loss: 0.5957 - background_loss: 0.3953 - close_loss: 0.4609 - ink_mark_loss: 0.4014 - pixelation_loss: 0.2561 - washed_out_loss: 0.0569 - blurred_loss: 0.1711 - sh_head_loss: 0.0799 - mouth_loss: 0.0335 - veil_loss: 0.3235 - red_eyes_loss: 0.2378 - flash_lenses_loss: 0.0498 - dark_glasses_loss: 0.4591 - l_away_loss: 0.2317 - frame_eyes_loss: 0.4481 - hair_eyes_loss: 0.3273 - eyes_closed_loss: 0.1897 - frames_heavy_loss: 0.0117 - sh_face_loss: 0.4977 - skin_tone_loss: 0.0522 - light_loss: 0.2430 - hat_loss: 0.3363 - rotation_loss: 0.4046 - reflection_loss: 0.2892 - background_accuracy: 0.8385 - close_accuracy: 0.7841 - ink_mark_accuracy: 0.8172 - pixelation_accuracy: 0.8904 - washed_out_accuracy: 0.9842 - blu

/home/guilherme/data2/anaconda3/envs/icao_nets_training/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/50
77/77 [==============================] - 341s 4s/step - loss: 0.3771 - background_loss: 0.2734 - close_loss: 0.3881 - ink_mark_loss: 0.2848 - pixelation_loss: 0.0961 - washed_out_loss: 0.0386 - blurred_loss: 0.0524 - sh_head_loss: 0.0177 - mouth_loss: 0.0101 - veil_loss: 0.1891 - red_eyes_loss: 0.1465 - flash_lenses_loss: 0.0109 - dark_glasses_loss: 0.3229 - l_away_loss: 0.1422 - frame_eyes_loss: 0.3501 - hair_eyes_loss: 0.2423 - eyes_closed_loss: 0.1139 - frames_heavy_loss: 6.5736e-05 - sh_face_loss: 0.3810 - skin_tone_loss: 0.0197 - light_loss: 0.1848 - hat_loss: 0.1685 - rotation_loss: 0.2203 - reflection_loss: 0.1173 - background_accuracy: 0.8900 - close_accuracy: 0.8332 - ink_mark_accuracy: 0.8817 - pixelation_accuracy: 0.9641 - washed_out_accuracy: 0.9892 - blurred_accuracy: 0.9807 - sh_head_accuracy: 0.9953 - mouth_accuracy: 0.9972 - veil_accuracy: 0.9186 - red_eyes_accuracy: 0.9409 - flash_lenses_accuracy: 0.9966 - dark_glasses_accuracy: 0.8598 - l_away_accuracy: 0.9

# Plots

In [ ]:
runner.draw_training_history()

# Load Best Model

In [ ]:
runner.load_best_model()

# Saving Trained Model

In [ ]:
runner.save_model()

# Test Trained Model

## Validation Split

In [ ]:
runner.set_model_evaluator_data_src(DataSource.VALIDATION)
runner.test_model(verbose=False)

## Test Split

In [ ]:
runner.set_model_evaluator_data_src(DataSource.TEST)
runner.test_model(verbose=False)

# Visualize Model Classification

# Finishing Experiment Manager

In [ ]:
runner.finish_experiment()